Load Dataset and do a sanity check:

In [22]:
#Import Libraries 
import numpy as np
import torch 
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms 
from PIL import Image 
import sys, pathlib
import torch.nn as nn
from Data_Loader import UnitCellDS
from Data_Loader import load_bandgap_data
import torch.optim as optim


dataset = load_bandgap_data('bandgap_data.mat', resize=64)

#Sanity Check
print(f"Total samples: {len(dataset)}")
x_sample, y_sample = dataset[78]
print(f"Image Shape: {x_sample.shape}")
print(f"Lable shape: {y_sample.shape}")
print(f"Label values: {y_sample}")





Total samples: 32768
Image Shape: torch.Size([1, 64, 64])
Lable shape: torch.Size([3])
Label values: tensor([1, 1, 1])


Create a Simple Wrapper 

In [23]:
#Simple class will have only index label 0 or 1
class SimpleDataset(torch.utils.data.Dataset):
    "Wrapper to just use one frequency range"
    def __init__(self, base_dataset, freq_idx=0):
        self.base_dataset = base_dataset
        self.freq_idx = freq_idx
    
    def __len__(self):
        return len(self.base_dataset)
    
    def __getitem__(self, idx):
        x, y = self.base_dataset[idx]
        return x, y[self.freq_idx]
    
simple_dataset = SimpleDataset(dataset, freq_idx=0)
x, y = simple_dataset[45]
print(f"Single Label: {y}")

Single Label: 1


Create a very simple CNN

In [24]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        #3 convolutional layers + 2 fully connected layers 
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.relu = nn.ReLU()

        self.fc1 = nn.Linear(64 * 8 * 8, 128)
        self.fc2 = nn.Linear(128, 2) #Binary for this case 

    def forward(self, x):
        x = self.pool(self.relu(self.conv1(x)))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(-1, 64 * 8 * 8) # Flatten 
        x = self.relu(self.fc1(x))
        x = self.fc2(x)
        return x

model = SimpleCNN()
print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")

    

Model parameters: 547,970


Split Data and create loaders

In [25]:
# 80/10/10 train test val split 

train_size = int(0.8 * len(simple_dataset))
val_size = int(0.1 * len(simple_dataset))
test_size = len(simple_dataset) - train_size - val_size

train_ds, val_ds, test_ds = random_split(simple_dataset, [train_size, val_size, test_size], generator=torch.manual_seed(42))

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

print(f"Train:{len(train_loader)} batches")
print(f"Val: {len(val_loader)} batches")




Train:820 batches
Val: 103 batches


Set up training loop: 

In [26]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"using device: {device}")

model = SimpleCNN().to(device)
criterion = nn.CrossEntropyLoss() #For Binary Classification
optimizer = optim.Adam(model.parameters(), lr=0.01)\

#Track Metrics
train_losses = []
val_losses = []
train_accs = []
val_accs = []

num_epochs = 20

print("Starting Traing ..\n")

for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    train_correct = 0
    train_total = 0

    for batch_idx, (images, labels) in enumerate(train_loader):
        #move to device 
        images = images.to(device)
        labels = labels.to(device)

        #Perform forward pass
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        #Backward pass
        loss.backward()
        optimizer.step()

        #Update Metrics
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        train_total += labels.size(0)
        train_correct += (predicted ==  labels).sum().item()

    avg_train_loss = train_loss / len(train_loader)
    train_acc = 100 * train_correct / train_total
    train_losses.append(avg_train_loss)
    train_accs.append(train_acc)

    #Validate Model
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad(): #no gradient computation during validation 
        for images, labels in val_loader:
            images = images.to(device)
            labels = labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            val_total += labels.size(0)
            val_correct += (predicted == labels).sum().item()
    
    #Calc avg val metrics
    avg_val_loss = val_loss / len(val_loader)
    val_acc = 100 * val_correct / val_total
    val_losses.append(avg_val_loss)
    val_accs.append(val_acc)

    print(f"Epoch [{epoch+1}/{num_epochs}]")
    print(f"  Train Loss: {avg_train_loss:.4f} | Train Acc: {train_acc:.2f}%")
    print(f"  Val Loss:   {avg_val_loss:.4f} | Val Acc:   {val_acc:.2f}%")
    print()

print("Training Complete")

torch.save(model.state_dict(), 'bandgap_cnn_simple.pth')
print("model saved as 'bandgap_cnn_simple.pth'")




using device: cpu
Starting Traing ..

Epoch [1/20]
  Train Loss: 0.3220 | Train Acc: 85.80%
  Val Loss:   0.2690 | Val Acc:   88.25%

Epoch [2/20]
  Train Loss: 0.2317 | Train Acc: 90.23%
  Val Loss:   0.2137 | Val Acc:   90.75%

Epoch [3/20]
  Train Loss: 0.2130 | Train Acc: 91.00%
  Val Loss:   0.1961 | Val Acc:   91.33%

Epoch [4/20]
  Train Loss: 0.2009 | Train Acc: 91.94%
  Val Loss:   0.2103 | Val Acc:   91.45%

Epoch [5/20]
  Train Loss: 0.1918 | Train Acc: 92.06%
  Val Loss:   0.2036 | Val Acc:   91.36%

Epoch [6/20]
  Train Loss: 0.1859 | Train Acc: 92.36%
  Val Loss:   0.2108 | Val Acc:   91.33%

Epoch [7/20]
  Train Loss: 0.1825 | Train Acc: 92.34%
  Val Loss:   0.1882 | Val Acc:   92.31%

Epoch [8/20]
  Train Loss: 0.1791 | Train Acc: 92.63%
  Val Loss:   0.1850 | Val Acc:   92.70%

Epoch [9/20]
  Train Loss: 0.1763 | Train Acc: 92.79%
  Val Loss:   0.1888 | Val Acc:   92.28%

Epoch [10/20]
  Train Loss: 0.1778 | Train Acc: 92.54%
  Val Loss:   0.1828 | Val Acc:   92.28%

E